In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-14"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8145,2024-10-14,Eua Nba,20:00,Indiana Pacers,Memphis Grizzlies,1.34,3.01,234.5,1.83,1.83,-7.5,1.96,1.72,djIr3JrF,0.746269,0.332226,0.546448,0.546448,0.078495,0.6,0.8,NaN,NaN,316.918,160.846771,0.507534,196.900,49.888512,0.253370,198.634,301.272,168.15,280.24,0.0,0.0,0.0,0.0,0.542928,0.000000,0.092231,-0.89,-0.178,-1.910112,0.636147,0.8,0.163853,2.97,0.594,3.383838,0.358991,0.5,0.141009
8146,2024-10-14,Eua Nba,20:30,Atlanta Hawks,Philadelphia 76ers,1.86,1.80,230.5,1.83,1.83,-1.5,2.03,1.67,llcNeyyk,0.537634,0.555556,0.546448,0.546448,0.093190,0.6,1.0,NaN,NaN,183.312,56.726451,0.309453,222.150,35.468597,0.159661,206.716,222.150,273.79,256.32,0.0,0.0,0.0,0.0,0.023184,0.000000,0.137599,-0.81,-0.162,-5.308642,0.544100,0.5,-0.044100,1.32,0.264,3.030303,0.458009,0.6,0.141991
8147,2024-10-14,Eua Nba,20:30,Brooklyn Nets,Washington Wizards,1.67,2.02,218.5,1.80,1.86,-3.5,1.96,1.72,hIBi1cDR,0.598802,0.495050,0.555556,0.537634,0.093852,0.4,0.0,NaN,NaN,198.088,56.428423,0.284865,213.116,86.198538,0.404468,185.384,273.774,196.20,347.80,0.0,0.0,0.0,0.0,0.134139,0.023184,0.092231,1.86,0.372,1.801075,0.529318,0.5,-0.029318,2.04,0.408,2.500000,0.332558,0.4,0.067442
8148,2024-10-14,Eua Nba,21:00,Milwaukee Bucks,Chicago Bulls,1.42,2.59,225.5,1.83,1.83,-6.5,1.96,1.72,rmjWgF51,0.704225,0.386100,0.546448,0.546448,0.090326,0.4,0.6,NaN,NaN,240.916,84.670858,0.351454,192.492,52.278048,0.271586,222.554,210.372,175.44,273.76,0.0,0.0,0.0,0.0,0.412626,0.000000,0.092231,1.58,0.316,1.329114,0.584647,0.7,0.115353,-0.86,-0.172,-9.244186,0.435101,0.5,0.064899
8149,2024-10-14,Eua Nba,23:30,Los Angeles Clippers,Dallas Mavericks,1.69,2.09,216.5,1.91,1.91,-1.5,1.83,1.90,AgrIHIc8,0.591716,0.478469,0.523560,0.523560,0.070185,0.4,0.6,NaN,NaN,205.128,95.397025,0.465061,187.920,44.844331,0.238635,172.206,259.232,166.65,331.08,0.0,0.0,0.0,0.0,0.149652,0.000000,0.026540,1.73,0.346,1.994220,0.537887,0.5,-0.037887,4.39,0.878,1.241458,0.495189,0.9,0.404811
8150,2024-10-14,Europa Eurocopa,14:00,Besiktas,Ulm,1.40,2.80,166.5,1.83,1.87,-7.5,1.97,1.64,KvzVjVDN,0.714286,0.357143,0.546448,0.534759,0.071429,0.4,0.6,NaN,NaN,148.260,40.507700,0.273221,192.186,38.642369,0.201068,143.742,173.496,138.58,212.44,0.0,0.0,0.0,0.0,0.471405,0.015289,0.129277,-0.75,-0.150,-2.666667,0.678418,0.7,0.021582,3.50,0.700,2.571429,0.552927,0.7,0.147073
8151,2024-10-14,Bulgária Nbl,13:15,Beroe,Academic Plovdiv,1.49,2.38,163.5,1.83,1.83,-5.5,2.01,1.68,ryGdpGPG,0.671141,0.420168,0.546448,0.546448,0.091309,0.8,0.6,NaN,NaN,258.784,117.436780,0.453802,201.324,161.084132,0.800124,182.968,413.934,225.40,145.27,0.0,0.0,0.0,0.0,0.325233,0.000000,0.126474,-2.24,-0.448,-1.093750,0.000000,0.0,0.000000,-3.35,-0.670,-2.059701,0.351271,0.1,-0.251271
8152,2024-10-14,China Cba,08:35,Fujian,Tianjin,2.19,1.65,216.5,1.81,1.89,1.5,2.00,1.70,ADOA68o3,0.456621,0.606061,0.552486,0.529101,0.062682,0.6,0.6,NaN,NaN,269.966,204.476196,0.757415,307.474,242.135244,0.787498,188.066,297.174,258.72,110.92,0.0,0.0,0.0,0.0,0.198874,0.030578,0.114666,0.96,0.192,6.197917,0.447815,0.5,0.052185,2.89,0.578,1.124567,0.436222,0.3,-0.136222
8153,2024-10-14,China Cba,08:35,Qingdao,Guangzhou,1.14,5.26,196.5,1.84,1.86,-12.5,1.88,1.80,WSCJ4nGF,0.877193,0.190114,0.543478,0.537634,0.067307,0.2,0.4,NaN,NaN,262.912,120.007698,0.456456,176.216,22.479142,0.127566,161.782,205.204,217.80,161.71,0.0,0.0,0.0,0.0,0.910400,0.007644,0.030744,-1.47,-0.294,-0.476190,0.628088,0.8,0.171912,-0.81,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
